https://blog.sionic.ai/Finetuning_Llama#1f6c7a1280af47c6a4a6f69022d3c3fd

In [43]:
!pip uninstall -y accelerate scipy tensorboardX peft bitsandbytes transformers trl torch
!pip uninstall -y adapter-transformers huggingface-hub torch datasets accelerate bitsandbytes
!pip install -q adapter-transformers huggingface-hub datasets
!pip install -q accelerate==0.21.0 scipy tensorboardX peft==0.4.0 bitsandbytes==0.40.2 transformers==4.32.0 trl==0.4.7 tensorboardX torch==2.2.0
!pip uninstall huggingface-hub -y
!pip install huggingface-hub==0.15.1
!pip install transformers --upgrade
!pip install datasets --upgrade

Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
Found existing installation: scipy 1.12.0
Uninstalling scipy-1.12.0:
  Successfully uninstalled scipy-1.12.0
Found existing installation: tensorboardX 2.6.2.2
Uninstalling tensorboardX-2.6.2.2:
  Successfully uninstalled tensorboardX-2.6.2.2
Found existing installation: peft 0.4.0
Uninstalling peft-0.4.0:
  Successfully uninstalled peft-0.4.0
Found existing installation: bitsandbytes 0.40.2
Uninstalling bitsandbytes-0.40.2:
  Successfully uninstalled bitsandbytes-0.40.2
Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: trl 0.4.7
Uninstalling trl-0.4.7:
  Successfully uninstalled trl-0.4.7
Found existing installation: torch 2.2.0
Uninstalling torch-2.2.0:
  Successfully uninstalled torch-2.2.0
Found existing installation: adapter-transformers 3.2.1.post0
Uni

In [53]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

ImportError: cannot import name 'HfFileSystem' from 'huggingface_hub' (/home/dacon_project/myenv/lib/python3.10/site-packages/huggingface_hub/__init__.py)

In [3]:
# Hugging Face 허브에서 훈련하고자 하는 모델을 가져와서 이름 지정
model_name = '42dot/42dot_LLM-SFT-1.3B'

# instruction 데이터 세트 설정
dataset_name = '/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Data/dacon증강데이터/train가공증강/엑셀gpt데이터증강-train가공.csv'

# fine-tuning(미세 조정)을 거친 후의 모델에 부여될 새로운 이름을 지정하는 변수
new_model = "/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Checkpoints/10epoch_Qlora"

Step 3. LoRA (Low-Rank Adaptation) 파라미터 설정

가중치를 조정하여 더 좋은 성능을 내어 추가적인 작업에 더 잘 맞도록 하는 기법이다. 이는 모델의 전체 구조를 변경하지 않으면서도 효율적으로 모델을 조정할 수 있다는 장점이 있다.
위에서 peft 라이브러리에서 Import한 LoraConfig 에 할당할 파라미터에 대해 하나씩 살펴보자.
먼저 lora_r 은 정보의 손실과 효율성 사이에 균형을 맞추기 위해 여러 시도를 통해 결정할 수 있는 파라미터이다. 모델의 가중치 행렬을 두 개의 low-rank 행렬의 곱으로 근사하기 때문에, lora_r 의 값이 커질 수록 더 많은 정보를 유지하면서 모델이 훈련 데이터에 과도하게 적응하게 하여 오버피팅의 가능성을 높인다. 한 편으로 lora_r 의 값이 낮을 수록 정보가 손실되며 모델이 데이터의 복잡한 특성을 포착할 가능성을 낮춘다.
또한 lora_alpha 는 LoRA 기업을 적용할 때 가중치에 곱해지는 scaling factor로서 원래의 모델에 대비하여 LoRA 가중치에 얼마나 영향을 미칠 지를 결정한다.
마지막으로 lora_dropout 파라미터는 전통적인 머신러닝에서 과적합을 방지하기 위하여 훈련 과정에서 네트워크의 일부 가중치를 0으로 설정하는 방법을 적용하기 위하여 쓰인다. 가중치 행렬을 조정할 때 low-rank matrices로 가중치 행렬을 근사하는 데 이 때 LoRA 변화량을 모델에 적용하기 전에 일부 원소를 임의로 0으로 만드는 방법이다.

In [4]:
# LoRA에서 사용하는 low-rank matrices 어텐션 차원을 정의. 여기서는 64로 설정
# 값이 크면 클수록 더 많은 수정이 이루어지며, 모델이 더 복잡해질 수 있음
lora_r = 64

# LoRA 적용 시 가중치에 곱해지는 스케일링 요소. 여기서는 16으로 설정
# LoRA가 적용될 때 원래 모델의 가중치에 얼마나 영향을 미칠지 결정. 높은 값은 가중치 조정의 강도를 증가시킴
lora_alpha = 16

# Dropout probability for LoRA layers   # LoRA 층에 적용되는 드롭아웃 확률. 여기서는 0.1 (10%)로 설정
lora_dropout = 0.1 # 일부 네트워크 연결을 무작위로 비활성화하여 모델의 강건함에 기여

Step 4.  bitsandbytes 파라미터 설정
앞서 언급했듯이 bitsandbytes 는 QLoRA 기법을 적용하기 위하여 사용되는 8비트 양자화 라이브러리이다. 해당 라이브러리를 통해 양자화 관련 설정값을 지정할 수 있다. 예를 들어 4비트 베이스 모델을 로딩한다던지, 4비트 모델의 연산 과정에서 사용할 데이터 타입을 지정한다던지, 양자화의 유형을 명시한다던지의 세부적인 설정을 진행할 수 있다.

In [5]:
# 4-bit precision 기반의 모델 로드
use_4bit = True

# 4비트 기반 모델에 대한 dtype 계산
bnb_4bit_compute_dtype = "float16"

# 양자화 유형(fp4 또는 nf4)
bnb_4bit_quant_type = "nf4"

# 4비트 기 모델에 대해 중첩 양자화 활성화(이중 양자화)
use_nested_quant = False

# Step 5. TrainingArguments 파라미터 설정

Trainer는 허깅페이스에서 제공하는 라이브러리로 모델의 학습부터 평가까지 한 번에 해결할 수 있는 API를 제공한다.
아래는 Train에 필요한 파라미터를 정의한 것으로, Optimizer의 종류와 Learing Rate, Epoch 수, Scheduler와 Half Precision의 사용 여부 등에 대해 지정할 수 있다.

num_train_epochs 파라미터를 통해 모델이 전체 데이터셋을 몇 번 반복하여 학습할 지를 지정할 수 있고, per_device_train_batch_size 를 통해 각 GPU에서 한 번에 처리할 데이터의 양을 나타낼 수도 있다.
여기에서는 모두 1로 지정하여 한 번에 하나의 데이터만 처리하도록 하였다. gradient_accumulation_steps 파라미터를 통해 기울기를 갱신하기 전에 몇 번의 기울기 업데이트를 축적할 지를 결정하는데, 여기에서는 1로 설정되어 있어 매 스텝마다 기울기를 갱신한다.

마지막으로 gradient_checkpointing 파라미터를 통해 메모리 사용을 최적화할 수 있다. 대규모 모델을 사용할 때 특히 유용한 파라미터로서 필요할 때만 특정 계층의 기울기를 저장하고 나머지는 버려 메모리의 부담을 줄인다.

max_grad_norm 파라미터를 통해 모델이 데이터로부터 학습하는 속도를 조절할 것이다. 기울기가 과도하게 커져서 발생할 수 있는 gradient exploding 문제 등을 방지할 수 있도록 기울기의 최대 크기를 설정한다.

그리고 이번 실습에서 Adam 옵티마이저를 사용할 것인데 이 때 learning_rate 를 보수적으로 잡아 모델이 데이터로부터 학습하는 속도를 적절히 늦추도록 할 것이다. 마지막으로 weight_decay 값을 적절히 주어 모델의 가중치가 너무 큰 값을 가지지 않도록 함으로서 오버피팅 현상을 해소할 수 있다.

스케줄러는 Cosine Decay를 사용하여 안정적으로 끊김없이 Loss가 감소하도록 할 것이다.

In [6]:
#모델이 예측한 결과와 체크포인트가 저장될 출력 디렉터리
output_dir = "./results"

# 훈련 에포크 수
num_train_epochs = 1

# fp16/bf16 학습 활성화(A100으로 bf16을 True로 설정)
fp16 = False
bf16 = False

# 훈련용 배치 크기
per_device_train_batch_size = 1

# 평가용 배치 크기
per_device_eval_batch_size = 1

# 그래디언트를 누적할 업데이트 스텝 횟수
gradient_accumulation_steps = 1

# 그래디언트 체크포인트 활성화
gradient_checkpointing = True


# 그래디언트 클리핑을 위한 최대 그래디언트 노름을 설정.
# 그래디언트 클리핑은 그래디언트의 크기를 제한하여 훈련 중 안정성을 높임.
# Maximum gradient normal (그래디언트 클리핑) 0.3으로 설정
max_grad_norm = 0.3

# 초기 학습률 AdamW 옵티마이저
learning_rate = 2e-6

# bias/LayerNorm 가중치를 제외하고 모든 레이어에 적용할 Weight decay 값
weight_decay = 0.001

# 옵티마이저 설정
optim = "paged_adamw_32bit"

# 학습률 스케줄러의 유형 설정, 여기서는 코사인 스케줄러 사용
lr_scheduler_type = "cosine"

# 훈련 스텝 수(num_train_epochs 재정의)
max_steps = -1

# (0부터 learning rate까지) 학습 초기에 학습률을 점진적으로 증가시키 linear warmup 스텝의 Ratio
warmup_ratio = 0.03

# 시퀀스를 동일한 길이의 배치로 그룹화, 메모리 절약 및 훈련 속도를 높임
group_by_length = True

# X 업데이트 단계마다 체크포인트 저장
save_steps = 25

# 매 X 업데이트 스텝 로그
logging_steps = 25

# Step 6. SFT 파라미터 값 설정

이전에 준비했던 데이터셋을 바탕으로 Supervised Fine-Tuning (SFT) 를 진행해보자. 우리는 SFTTrainer로 Import한 라이브러리를 사용할 것이다.
SFT는 라벨러들이 제작한 프롬프트 데이터셋을 이용하여 베이스 모델을 지도학습 바탕으로 파인 튜닝하는 기법으로, 주어진 일정 토큰에 대한 다음 토큰을 예측하는 형식으로 진행된다. 우리의 데이터셋으로 치면 주어진 질문에 대한 답변을 유추하는 형태가 될 것이다.
max_seq_length 는 입력 시퀀스에 대한 최대 사이즈를 의미한다. 예를 들어 문장 2개가 합쳐질 때 maximum sequence가 어느 정도일 지를 결정한다.
packing 이란 훈련 과정에서의 효율성을 높이기 위해 복수 개의 예시 문장을 하나의 Input 시퀀스로 넣어주는 기법을 의미한다.
device_map 을 통해서 몇 번 GPU를 로드할 지 지정할 수 있다. 본인은 학습 환경이 GPU 1대였기 때문에 0번 GPU를 로드하도록 설정하였다.

In [7]:
# 최대 시퀀스 길이 설정
max_seq_length = None

# 동일한 입력 시퀀스에 여러 개의 짧은 예제를 넣어 효율성을 높일 수 있음
packing = False

# GPU 0 전체 모델 로드
device_map = {"": 0}

# Step 7. 데이터 세트 로딩과 데이터 타입 결정
이제 위에서 알아보았던 파라미터를 주입해보도록 하자. 먼저 SFT를 진행하기 위한 데이터셋을 로드한다. 그리고 BitsAndBytesConfig 인스턴스를 생성할 때 앞서 살펴보았던 데이터 타입을 주입하도록 하자.

In [8]:
import pandas as pd
df = pd.read_csv(dataset_name)
# 특정 문자열을 빈 문자열로 대체 (예: '"문자', '"이 문자' 제거)
df.dropna(inplace=True)
df = df.apply(lambda x: x.str.replace('"', '').str.replace('"', ''), axis=1)
df.to_csv(dataset_name, index=False)

In [9]:
import pandas as pd
from tqdm import tqdm
# 데이터 로드
data = pd.read_csv(dataset_name)

# 필요한 열을 문자열로 변환
for col in ['Q1', 'Q2', 'A1', 'A2']:
    data[col] = data[col].astype(str)

# 데이터 포맷팅 및 토크나이징
formatted_data = {}
list1 = []
list2 = []
for _, row in tqdm(data.iterrows()):
    for num in range(1,3):
        list1.append(row[f'Q{num}'])
        list2.append(row[f'A{num}'])
print('Done.')
formatted_data['Q'] = list1
formatted_data['A'] = list2
df = pd.DataFrame(formatted_data)
df.to_csv('/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Data/load_dataset/Q_A_utf-8.csv', encoding='utf-8', errors='replace', index=False)

642it [00:00, 16217.14it/s]

Done.


# Step 8. GPU 호환성 확인
현재 GPU가 bfloat16 형태를 지원하는 지 확인한다. 만약 GPU의 CUDA 버전이 8 이상이라면 해당 데이터 타입을 지원하는 것이다.
bfloat16이란 모델의 트레이닝 속도를 높일 수 있는 데이터 타입으로서, 16비트 부동 소수점 형식을 나타낸다. 물론 32비트 부동 소수점 형식보다는 정확도가 비교적 떨어지지만 메모리 요구 사항이 적다는 점에서 모델 학습에 유용하다고 할 수 있다.

In [10]:
from sklearn.model_selection import train_test_split
# Load dataset
load_dataset_path = '/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Data/load_dataset'
dataset = load_dataset(load_dataset_path, split="train")

# 훈련 데이터셋을 훈련 세트와 검증 세트로 분할
train_dataset, validation_dataset = dataset.train_test_split(test_size=0.1).values()


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # 모델을 4비트로 로드할지 여부를 결정
    bnb_4bit_quant_type=bnb_4bit_quant_type, # 양자화 유형을 설정
    bnb_4bit_compute_dtype=compute_dtype,  # 계산에 사용될 데이터 타입을 설정
    bnb_4bit_use_double_quant=use_nested_quant, # 중첩 양자화를 사용할지 여부를 결정
)

# 만약 GPU가 최소한 버전 8 이상이라면 (major >= 8) bfloat16을 지원한다고 메시지를 출력.
# bfloat16은 훈련 속도를 높일 수 있는 데이터 타입.
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
train_dataset, validation_dataset

(Dataset({
     features: ['Q', 'A'],
     num_rows: 1155
 }),
 Dataset({
     features: ['Q', 'A'],
     num_rows: 129
 }))

In [12]:
train_dataset['Q'][0]

'인테리어에서 조명을 선택할 때 고려해야 할 중요한 요소는 무엇인가요?'

평가지표 함수(준비중)

In [13]:
# from datasets import load_metric
# import numpy as np
# import torch

# # 평가 지표 로드
# bleu_metric = load_metric("bleu")
# rouge_metric = load_metric("rouge")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # 모델 출력에서 텍스트로 디코딩
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # 레이블(참조 텍스트)에서 텍스트로 디코딩
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # BLEU 점수 계산
#     bleu_result = bleu_metric.compute(predictions=[decoded_preds], references=[[decoded_labels]])

#     # ROUGE 점수 계산
#     rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

#     # PPL 계산 (이 부분은 모델과 데이터셋에 따라 수정이 필요할 수 있습니다.)
#     # 예시 코드에서는 PPL 계산을 위한 로직이 생략되어 있으며, 이는 모델의 로그 확률을 기반으로 직접 계산해야 합니다.

#     # 결과 집계
#     results = {
#         "bleu": bleu_result["bleu"],
#         "rouge-1": rouge_result["rouge1"].mid.fmeasure,
#         "rouge-2": rouge_result["rouge2"].mid.fmeasure,
#         "rouge-l": rouge_result["rougeL"].mid.fmeasure,
#         # "ppl": ppl 계산 결과 추가
#     }
#     return results


In [14]:
# from transformers import Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",

#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     report_to="none",  # 텐서보드 등의 리포팅을 사용하지 않는 경우
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
# )

# trainer.train()


# Step 9. 베이스 모델 로딩
이제 사전 훈련된 베이스 모델과 토크나이저를 로드한 다음 LoRA 연산을 적용해보자.

LLM(Large Language Model)과 같은 대규모 언어 모델의 평가 지표는 모델이 수행하는 작업의 유형에 따라 달라질 수 있습니다. LLM은 다양한 자연어 처리(NLP) 작업에 활용될 수 있으며, 각 작업마다 적합한 평가 지표가 있습니다. 여기 몇 가지 일반적인 작업과 그에 적합한 평가 지표를 소개합니다:

### 1. 텍스트 생성 (Text Generation)
- **Perplexity (PPL)**: 모델이 텍스트를 얼마나 잘 생성하는지를 측정하는 지표입니다. 낮은 PPL 값은 모델이 데이터를 잘 이해하고 있음을 의미합니다.
- **BLEU, ROUGE, METEOR**: 이러한 지표들은 주로 기계 번역에서 사용되지만, 생성된 텍스트와 참조 텍스트 간의 유사성을 측정하는 데 사용될 수 있습니다. 텍스트 생성의 정확성과 유창성을 평가하는 데 유용합니다.

### 2. 텍스트 분류 (Text Classification)
- **정확도 (Accuracy)**: 전체 예측 중 올바른 예측의 비율입니다. 간단하고 직관적인 지표입니다.
- **F1 점수**: 정밀도와 재현율의 조화 평균으로, 클래스 간 불균형이 있는 경우 유용한 지표입니다.
- **AUC-ROC**: 모델의 성능을 여러 임계값에서 평가하며, 클래스 간 불균형이 심할 때 유용합니다.

### 3. 질의 응답 (Question Answering)
- **정확도 (Accuracy)**: 특정 기준(예: 정확한 답변, 완전 일치 등)에 따른 모델의 정확도 측정입니다.
- **F1 점수**: 예측된 답변과 실제 답변 간의 토큰 일치를 기반으로 한 지표로, 부분적인 일치도 고려할 수 있습니다.

### 4. 요약 (Summarization)
- **ROUGE**: 생성된 요약과 실제 요약 간의 오버랩을 측정합니다. ROUGE-N(단어 n-gram 정밀도와 재현율) 및 ROUGE-L(가장 긴 공통 부분 문자열) 등 여러 변형이 있습니다.

### 5. 기계 번역 (Machine Translation)
- **BLEU**: 기계 번역의 품질을 측정하는 대표적인 지표로, 번역된 문장과 참조 번역 사이의 단어 n-gram 정밀도를 기반으로 합니다.
- **METEOR**: BLEU의 일부 단점을 보완하기 위해 개발되었으며, 동의어 매칭과 문장 구조를 고려합니다.

LLM을 평가할 때는 모델의 사용 목적과 작업 유형을 고려하여 적절한 평가 지표를 선택해야 합니다. 또한, 모델의 성능을 종합적으로 이해하기 위해 여러 지표를 함께 사용하는 것이 좋습니다.

In [15]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 동일한 batch 내에서 입력의 크기를 동일하기 위해서 사용하는 Padding Token을 End of Sequence라고 하는 Special Token으로 사용한다.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training. Padding을 오른쪽 위치에 추가한다.

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM", # 파인튜닝할 태스크를 Optional로 지정할 수 있는데, 여기서는 CASUAL_LM을 지정하였다.
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="steps", # 또는 "steps"
    eval_steps=25, # 100 스텝마다 검증, "evaluation_strategy"가 "steps"일 경우에만 필요
    save_strategy="steps",
    save_steps=save_steps,
    load_best_model_at_end=True, # 훈련 종료 시 최적의 모델을 자동으로 로드
    metric_for_best_model="loss", # 최적의 모델을 결정할 때 사용할 메트릭
    save_total_limit=3,  # 저장할 체크포인트의 최대 개수 설정
    )

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,  # 평가 데이터셋 지정
    peft_config=peft_config,
    dataset_text_field="Q",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [25]:

!pip install accelerate==0.27.2
!pip install transformers --upgrade
!pip install Trl --upgrade
!pip uninstall accelerate -y
!pip uninstall bitsandbytes -y
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


Y
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: Trl
    Found existing installation: trl 0.4.7
    Uninstalling trl-0.4.7:
      Successfully uninstalled trl-0.4.7


Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
Found existing installation: bitsandbytes 0.40.2
Uninstalling bitsandbytes-0.40.2:
  Successfully uninstalled bitsandbytes-0.40.2
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
Y


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 13.3 MB/s eta 0:00:00


In [ ]:
trainer

In [ ]:
plus_epoch_model = "/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/Checkpoints/Test2_1epoch_Qlora"

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(plus_epoch_model)

Step,Training Loss,Validation Loss
25,3.940400,4.026704
50,4.206000,4.017161
75,3.947100,4.005848
100,4.340400,3.992542
125,3.638300,3.978784
150,4.282900,3.964014
175,3.869500,3.948381
200,4.425400,3.931107
225,3.672900,3.912733
250,4.072900,3.893239


# Step 10. 모델 훈련과 훈련된 모델 저장
이제 모델을 저장하면 bade_model 파라미터를 제외한 Adapter 부분만 저장이 될 것이다. 아래와 같이 adapter_config.json 파일이 보이면 성공이다.

훈련 과정에서 검증 손실(validation loss)이 일시적으로 상승한 후 다시 감소하는 현상은 여러 가지 이유로 발생할 수 있으며, 반드시 문제가 있는 것은 아닙니다. 이러한 현상을 이해하기 위해 몇 가지 가능한 시나리오를 고려해 볼 수 있습니다:

### 1. **모델이 아직 최적의 상태에 도달하지 않았을 때**
- 모델이 훈련 초기 단계에 있고 아직 데이터에 대해 충분히 학습하지 못한 경우, 검증 손실이 불규칙하게 움직일 수 있습니다. 이는 모델이 아직 최적의 매개변수를 찾아가는 과정에 있음을 나타냅니다.

### 2. **학습률(Learning Rate)이 높을 때**
- 학습률이 너무 높으면, 모델이 최적점을 넘어서는 경우가 발생할 수 있습니다. 이로 인해 손실이 일시적으로 증가할 수 있으며, 학습률을 점차 줄여가는 학습률 스케줄러를 사용하면 이 문제를 완화할 수 있습니다.

### 3. **미니 배치의 변동성 때문에**
- 미니 배치 학습을 사용할 때 각 배치의 데이터가 모델에 미치는 영향은 다를 수 있습니다. 특정 배치에서 어려운 또는 이상치(outlier)를 포함한 샘플이 있을 경우 일시적으로 손실이 증가할 수 있습니다.

### 4. **과적합(Overfitting) 초기 징후일 수 있음**
- 훈련 데이터에 과도하게 맞추어져서 검증 데이터에 대한 일반화 성능이 떨어지는 과정일 수 있습니다. 그러나 손실이 다시 감소한다면, 모델이 다시 일반화 성능을 회복하고 있는 것일 수 있습니다.

### 5. **정규화 기법의 영향**
- 드롭아웃(Dropout)이나 배치 정규화(Batch Normalization)와 같은 정규화 기법들이 모델에 적용되어 있을 경우, 이러한 기법들이 학습 과정에 변동성을 추가할 수 있으며, 이는 손실에 영향을 미칠 수 있습니다.

### 대응 방안
- **학습률 조정**: 학습률 스케줄러를 사용하여 학습률을 점차 줄여나가는 전략을 적용해보세요.
- **조기 종료(Early Stopping)**: 검증 손실이 일정 기간 동안 개선되지 않으면 훈련을 조기에 종료하고, 가장 낮은 검증 손실을 가진 모델을 선택합니다.
- **정규화 기법 조정**: 드롭아웃 비율이나 배치 정규화 파라미터 등을 조정해보세요.
- **데이터 검토**: 훈련 및 검증 데이터셋이 적절하게 구성되었는지 확인하고, 필요한 경우 데이터 전처리 과정을 조정해보세요.

검증 손실의 변동은 훈련 과정에서 자연스러운 현상일 수 있으며, 전체 훈련 과정을 통틀어 모델의 성능 향상 추세를 관찰하는 것이 중요합니다.



In [ ]:

# Save trained model
trainer.model.save_pretrained(new_model)

# 모델 불러오기

In [ ]:
import os  # os 모듈 운영체제와 상호 작용할 수 있는 기능을 제공
import torch # PyTorch 라이브러리로, 주로 딥러닝과 머신러닝 모델을 구축, 학습, 테스트하는 데 사용
from datasets import load_dataset  # 데이터셋을 쉽게 불러오고 처리할 수 있는 기능을 제공
from transformers import (
    AutoModelForCausalLM, # 인과적 언어 추론(예: GPT)을 위한 모델을 자동으로 불러오는 클래스
    AutoTokenizer, # 입력 문장을 토큰 단위로 자동으로 잘라주는 역할
    BitsAndBytesConfig, # 모델 구성
    HfArgumentParser,  # 파라미터 파싱
    TrainingArguments,  # 훈련 설정
    pipeline,  # 파이프라인 설정
    logging,  #로깅을 위한 클래스
)

# 모델 튜닝을 위한 라이브러리
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

model_name = '42dot/42dot_LLM-SFT-1.3B'
new_model = "/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/10epoch_Qlora"

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(base_model, new_model)

In [ ]:
model = model.merge_and_unload()

In [ ]:
pip install transformers --upgrade

In [ ]:
pip install accelerate --upgrade

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 동일한 batch 내에서 입력의 크기를 동일하기 위해서 사용하는 Padding Token을 End of Sequence라고 하는 Special Token으로 사용한다.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training. Padding을 오른쪽 위치에 추가한다.

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM", # 파인튜닝할 태스크를 Optional로 지정할 수 있는데, 여기서는 CASUAL_LM을 지정하였다.
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="Q",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
plus_epoch_model = "/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/15epoch_Qlora"

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(plus_epoch_model)

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


Step,Training Loss
25,4.423300
50,5.943900
75,4.515600
100,5.686600
125,4.492900
150,6.061800
175,4.657700
200,6.149500
225,4.868100
250,6.027500


In [ ]:
plus_epoch_model = "/content/drive/MyDrive/TeamProjects/Dacon_Project/Hansol/1.도배하자Project/10epoch_Qlora"
# Save trained model
trainer.model.save_pretrained(plus_epoch_model)

In [ ]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import pandas as pd
test = pd.read_csv('/mnt/h/내 드라이브/chatbot_project/Data/test.csv')
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [ ]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:

import warnings

# 모든 경고 메시지를 무시하도록 설정합니다.
warnings.filterwarnings("ignore")

# 기존에 학습된 모델을 불러옵니다. 여기서 AutoModelForCausalLM은 인과적 언어 모델을 위한 클래스입니다.
# 'model_name'은 불러올 모델의 이름을 지정합니다.
# 'low_cpu_mem_usage'는 CPU 메모리 사용을 최소화하도록 설정합니다.
# 'return_dict'는 모델의 출력을 딕셔너리 형태로 반환하도록 설정합니다.
# 'torch_dtype=torch.float16'는 모델 연산에서 사용할 데이터 타입을 16비트 부동소수점으로 설정합니다.
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)

# PeftModel 클래스를 이용하여 기존 모델에 새로운 모델을 추가합니다.
model = PeftModel.from_pretrained(base_model, plus_epoch_model)

# 모델의 리소스를 정리합니다.
model = model.merge_and_unload()

# 토크나이저를 로드합니다. 이는 텍스트를 모델이 이해할 수 있는 형태로 변환하는데 사용됩니다.
# 'trust_remote_code=True'는 원격 코드를 신뢰하고 사용하도록 설정합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 토크나이저의 패딩 토큰을 문장 종료 토큰으로 설정합니다.
tokenizer.pad_token = tokenizer.eos_token

# 패딩이 오른쪽에 적용되도록 설정합니다.
tokenizer.padding_side = "right"

# 텍스트 입력을 정의합니다. 사용자에게 정보를 요청하는 프롬프트입니다.
prompt = "도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 건축 전문가적 입장에서 300자 이내로 요약해서 답변해주세요."

# 주어진 텍스트를 토큰화합니다.
tokens = tokenizer(text=f"{prompt}\n답변:", return_tensors="pt")

# 모델을 GPU로 옮깁니다.
model = model.to('cuda:0')

# 토큰화된 데이터를 GPU로 옮깁니다.
tokens = tokens.to('cuda:0')


# 텍스트를 생성합니다. 이는 모델에 입력 토큰을 넣고, 지정된 설정에 따라 텍스트를 생성합니다.
generated_ids = model.generate(
    tokens["input_ids"],           # 모델에 입력할 토큰 ID들입니다. 이전에 토큰화한 결과가 여기에 사용됩니다.
    max_length=600,         # 생성할 텍스트의 최대 길이를 지정합니다. 이 경우 최대 350개의 토큰까지 생성될 수 있습니다.
    min_length=50,                 # 생성할 텍스트의 최소 길이를 지정합니다. 적어도 50개의 토큰은 생성될 것입니다.
    num_return_sequences=1,  # 생성할 텍스트 시퀀스의 수를 지정합니다. 여기서는 1개의 시퀀스만 생성하도록 설정했습니다.
    pad_token_id=tokenizer.eos_token_id,  # 패딩에 사용할 토큰 ID를 지정합니다. 여기서는 문장 종료 토큰(eos_token)의 ID를 사용합니다.
    eos_token_id=tokenizer.eos_token_id,  # 문장 종료 토큰의 ID를 지정합니다. 생성된 텍스트가 이 토큰을 만나면 생성을 중단합니다.
    no_repeat_ngram_size=3,      # 반복되는 n-gram의 크기를 지정합니다. 여기서는 3개의 연속된 토큰이 반복되지 않도록 설정합니다.
    temperature=0.3,                  # 생성 과정에서의 무작위성 정도를 지정합니다. 0으로 설정하면 가장 확률이 높은 토큰만 선택되어 매우 결정론적인 결과가 생성됩니다.
    # top_p=0.6,    #0과 1 사이의 값을 설정합니다. 0에 가까울수록 가장 높은 확률을 가진 토큰만 선택되어 반복 가능성이 높아지고, 1에 가까울수록 다양한 토큰이 선택되어 반복 가능성이 낮아집니다.
    # repetition_penalty=4.8 #0 이상의 값을 설정합니다. 값이 높을수록 동일한 단어 또는 토큰이 연속적으로 나타나는 것을 더욱 강하게 방지합니다.
)


# 생성된 텍스트를 디코딩합니다. 이는 생성된 토큰 ID를 실제 텍스트로 변환합니다.
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# 원하는 부분만 추출합니다.
start_token = "답변:"
start_idx = generated_text.find(start_token)
if start_idx != -1:
    # "답변:" 이후의 텍스트만 추출합니다.
    final_text = generated_text[start_idx + len(start_token):].strip()
else:
    # "답변:" 토큰이 없는 경우, 전체 텍스트를 사용합니다.
    final_text = generated_text

# 최종적으로 추출된 텍스트를 출력합니다.
print(final_text)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


녹은 자국은 건축물의 표면에 발생하는 문제 중 하나입니다. 이 문제는 주로 다음과 같은 원인으로 인해 발생합니다.

1. 열과 습기: 건축물은 열과 수분을 흡수하고 방출하는 역할을 합니다. 그러나 열과 과도한 습기는 표면의 접착력을 약화시키고 녹을 유발할 수 있습니다.
2. 화학 물질: 건축물에 사용되는 화학 물질은 표면에 녹을 일으킬 수 있습니다(예: 페인트, 접착제, 마감재 등).
3. 기계적 충격: 건축물의 구조적인 결함이나 기계적 손상은 표면에 충격을 가할 수 있으며, 이는 녹을 발생시킬 수 있습니다

녹을 해결하기 위해 다음과 같은 방법을 고려할 수 있지만, 이는 건축 전문가의 판단에 따라 다를 수 있습니다:

- 열과 화학 물질에 대한 보호: 건축물 표면에 보호 필름을 적용하여 열과 유해한 화학 물질의 영향을 최소화할 수 있도록 합니다.
- 기계적 손상 방지: 건축물이 기계적 충격을 받지 않도록 보호하고, 필요한 경우 보강재를 사용하여 구조적인 손상을 방지합니다.


In [ ]:
#템퍼레처 노 리핏 for i 구문 추가 해서 확인해보기


import warnings

# 모든 경고 메시지를 무시하도록 설정합니다.
warnings.filterwarnings("ignore")

# 기존에 학습된 모델을 불러옵니다. 여기서 AutoModelForCausalLM은 인과적 언어 모델을 위한 클래스입니다.
# 'model_name'은 불러올 모델의 이름을 지정합니다.
# 'low_cpu_mem_usage'는 CPU 메모리 사용을 최소화하도록 설정합니다.
# 'return_dict'는 모델의 출력을 딕셔너리 형태로 반환하도록 설정합니다.
# 'torch_dtype=torch.float16'는 모델 연산에서 사용할 데이터 타입을 16비트 부동소수점으로 설정합니다.
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)

# PeftModel 클래스를 이용하여 기존 모델에 새로운 모델을 추가합니다.
model = PeftModel.from_pretrained(base_model, new_model)

# 모델의 리소스를 정리합니다.
model = model.merge_and_unload()

# 토크나이저를 로드합니다. 이는 텍스트를 모델이 이해할 수 있는 형태로 변환하는데 사용됩니다.
# 'trust_remote_code=True'는 원격 코드를 신뢰하고 사용하도록 설정합니다.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 토크나이저의 패딩 토큰을 문장 종료 토큰으로 설정합니다.
tokenizer.pad_token = tokenizer.eos_token

# 패딩이 오른쪽에 적용되도록 설정합니다.
tokenizer.padding_side = "right"

# 텍스트 입력을 정의합니다. 사용자에게 정보를 요청하는 프롬프트입니다.
prompt = "어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요? 그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요? 건축 전문가적 입장에서 300자 이내로 요약해서 답변해주세요."

# 주어진 텍스트를 토큰화합니다.
tokens = tokenizer(text=f"{prompt}\n답변:", return_tensors="pt")

# 토큰화된 데이터를 GPU로 옮깁니다.
tokens = tokens.to('cuda:0')

# 모델을 GPU로 옮깁니다.
model = model.to('cuda:0')

import csv

# 결과를 저장할 딕셔너리 (키: temperature, 값: 해당 temperature에 대한 모든 ngram_size 결과 리스트)
results = {temp: [] for temp in [0, 0.1, 0.2, 0.3, 0.4]}

# 모든 temperature와 no_repeat_ngram_size 조합에 대해 반복
for temp in [0, 0.1, 0.2, 0.3, 0.4]:
    for ngram_size in [0, 1, 2, 3]:
        generated_ids = model.generate(
            tokens["input_ids"],
            max_length=512,
            min_length=50,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=ngram_size,
            temperature=temp
        )

        # 생성된 텍스트를 디코딩
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # "답변:" 이후의 텍스트만 추출
        start_idx = generated_text.find("답변:")
        final_text = generated_text[start_idx + len("답변:"):] if start_idx != -1 else generated_text

        # 결과 딕셔너리에 추가
        results[temp].append(final_text.strip())

# CSV 파일로 저장
with open("/mnt/g/라마/텀퍼레처_노리핏_테스트결과/몰딩.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # 헤더 작성
    header = ["Temperature", "N-Gram 0", "N-Gram 1", "N-Gram 2", "N-Gram 3"]
    writer.writerow(header)

    # 데이터 작성
    for temp, texts in results.items():
        row = [temp] + texts
        writer.writerow(row)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#test.csv 질문에 모두 답변 하는 코드

from tqdm import tqdm


test = pd.read_csv('/mnt/h/내 드라이브/chatbot_project/Data/test.csv')

# 텍스트 생성 설정

preds = []

for test_question in tqdm(test['질문']):

    tokens = tokenizer(f"{test_question} 건축 전문가적 입장에서 300자 이내로 요약해서 정확한 값으로 답변해주세요.\n답변:", return_tensors="pt")
    model = model.to('cuda:0')
    tokens = tokens.to('cuda:0')

    # 텍스트 생성
    generated_ids = model.generate(
        tokens["input_ids"],
        max_length=600,
        min_length=50,
        num_return_sequences = 1,  # 생성할 텍스트 시퀀스의 수
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        temperature=0.3
    )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # "답변:" 이후의 텍스트만 추출
    start_token = "답변:"
    start_idx = generated_text.find(start_token)
    if start_idx != -1:
        final_text = generated_text[start_idx + len(start_token):].strip()
    else:
        final_text = generated_text

    preds.append(final_text)
    print(f"질문: {test_question}\n답변: {final_text}\n")




  0%|          | 0/130 [00:00<?, ?it/s]

  1%|          | 1/130 [00:07<16:14,  7.55s/it]

질문: 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변: 방청 페인팅에는 다양한 종류가 있습니다. 일반적으로 사용되는 방청 도료는 다음과 같습니다.
1. 아연 페인트: 아연은 철과 알루미늄을 보호하는 데 효과적입니다. 주로 벽이나 천장에 사용됩니다.
2. 니켈 페인트 및 페인트 시리즈: 니켈은 철을 보호하고 녹을 방지하는 데 사용됩니다, 주로 벽과 천장에 사용되는 페인트입니다.
3. 몰딩 페인트와 몰딩 시리즈: 몰딩은 벽이나 지붕에 사용되는 페인테이션으로, 철과 목재 사이의 접착력을 향상시키는 데 효과적이죠.
원목사이징을 사용하는 것에는 몇 가지 단점이 있습니다.
첫째, 원목을 사용하는 것은 목재에 손상을 줄 수 있습니다. 목재의 결을 손상시키면 내구성이 저하될 수 있습니다
둘째, 목재 사이징은 시간이 오래 걸릴 수 있습니다, 목재를 고정하고 사이징하는 데 필요한 도구와 방법이 제한적일 수 있습니다.
셋째, 사이징을 할 때 목재 표면에 흠집이 생길 수 있습니다, 이로 인해 페인트가 제대로 도포되지 않을 수 있습니다



  2%|▏         | 2/130 [00:11<11:48,  5.53s/it]

질문: 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변: 도배지에 발생하는 녹은 자국 주된 원인 중 하나는 도배지 시공 시 사용되는 접착제의 품질과 관련이 있습니다. 접착제가 불충분하거나 부적절하게 사용되면 도배지가 제대로 고정되지 않아 녹이 발생할 수 있습니다.
해결 방법으로는 도배지의 접착제를 확인하고 필요한 경우 개선하는 것이 중요합니다. 또한, 도배지를 시공할 때 도배지와 벽면 사이의 공간을 충분히 확보하여 접착제 분진이 벽면에 부착되는 것을 방지하는 것이 좋습니다. 또한 도배지나 벽면에 먼지나 이물질이 쌓이지 않도록 주의해야 합니다.
이러한 방법을 통해 도배지에서 발생하는 녹을 효과적으로 방지할 수 있습니다



...

질문: 평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?
답변: 평지붕에 사용되는 평지형 평지퍼는 누수를 방지하기 위한 효과적인 도료입니다. 이 도료는 수성 도료로, 우수한 내구성과 방수 성능을 가지고 있어 평지벽의 누수를 효과적으로 방지할 수 있습니다.

첫째로, 수성 평지 퍼는 건조 시간이 짧아 작업 시간을 단축시킬 수 있습니다, 이는 작업 효율성을 향상시킵니다.
둘째로 수성 퍼는 내구성이 뛰어나며, 다양한 재질에 적용할 수 있어 다양한 건축 구조에 적용하기 적합합니다.
셋째로 평지 벽에 사용되는 수성 탄성 도료는 우수한 방수 성능으로 인해 누수를 예방할 수 있으며, 특히 습한 환경에서 효과적인 방수 효과를 제공합니다.
마지막으로, 수성의 평지 평지페는 내후성이 뛰어나고 내구성을 보장하기 때문에 장기간 사용해도 변색이나 손상이 적습니다.
따라서, 수성한 평지평지퍼를 사용하는 것은 평지부의 누수를 막기 위한 효과적인 방법입니다.



 99%|█████████▉| 129/130 [14:48<00:07,  7.53s/it]

질문: 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."
답변: 석고수정은 주로 석고보드의 결함으로 인해 발생하는 현상입니다. 석고의 수분이 증발하면서 석고판의 결함이 노출되어 석고 수정이 발생할 수 있습니다. 석고를 사용하는 건축 재료 중 석고 보드는 가장 일반적인 재료 중 하나입니다.
석고 수정을 해결하기 위해 가장 일반적인 방법은 석고판을 교체하는 것입니다. 석고가 수분을 흡수하면서 석고가 팽창하여 석고판이 결함 부위를 노출시킬 수 있습니다, 따라서 석고 판을 교체하면 결함을 덮어줄 수 있습니다 또한, 석고판에 결함이 있는 경우 석고 부스러기를 제거하는 것이 중요합니다. 석재 표면에 결함이 있을 경우 석재 부스러기가 석고에 흡수되어 석고를 부풀릴 수 있습니다 따라서 석재 결함을 제거하는 것도 석고수지 발생을 방지하는 데 도움이 됩니다.
또한, 석고를 사용할 때 주의해야 할 사항이 있습니다. 가장 중요한 것은 석고 사용 시 결함을 방지하기 위해 석고 발라야 하는 부위를 정확하게 설정하는 것입니다, 석고가 증발할 때 결함이 발생할 수 있으므로, 석재에 결함이 없는 부위에 석고를 발라주는 것이 중요하며, 석재의 결함을 확인하기 위해 정기적으로 석고 표면을 확인하는 것도 필요합니다.
따라서 석고수리 발생을 해결하기 위해서는 석고 패널 교체, 석부스러기 제거, 결함 방지 부위 설정, 정기적인 석고 확인 등의 방법을 사용하여 석고수의 발생을 예방할 수 있습니다



100%|██████████| 130/130 [14:54<00:00,  6.88s/it]

질문: 카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요?
답변: 카페트의 예상 수명은 일반적으로 10년에서 15년 정도입니다. 이는 일반적으로 사용되는 카페트의 재질과 디자인에 따라 다를 수 있습니다.

오리지널 징크의 장점은 무엇인가요?
오리징크는 내구성이 뛰어나고 오래 사용할 수 있는 징크 판넬의 종류입니다. 징크는 목재와 유사한 질감과 패턴을 제공하여 실내 공간을 아름답게 꾸밀 수 있습니다, 또한, 징크판넬은 내화성이 뛰어나며 화재 발생 시에도 안전합니다.
또한, 징크를 사용하는 경우 벽면에 균열이 생기는 것을 방지할 수 있습니다.,

위의 정보를 종합하면, 카페트의 예상 기대 수명이 10년 ~ 15년이며, 오리지날 징크가 내구성과 오래 사용할 수있는 장점이 있습니다.



In [ ]:
# 질문 답변 모두 저장
import pandas as pd

# DataFrame 생성
# 'test' 데이터프레임에서 '질문' 컬럼과 preds 리스트(답변)를 결합합니다.
df_preds = pd.DataFrame({'질문': test['질문'], '답변': preds})

# CSV 파일로 저장
csv_file_path = '/mnt/g/라마/전체답변/QLoRA_10epoch_inference02.csv'  # 저장할 경로 설정
df_preds.to_csv(csv_file_path, encoding='utf-8', index=False)  # 인덱스를 제외하고 csv 파일로 저장

csv_file_path  # 생성된 파일의 경로를 출력합니다.


'/mnt/g/라마/전체답변/QLoRA_10epoch_inference01.csv'

In [ ]:
#답변만 저장

import pandas as pd

# DataFrame 생성
df_preds = pd.DataFrame(preds, columns=['답변'])

# CSV 파일로 저장
csv_file_path = '/mnt/g/라마/전체답변/QLoRA_10epoch_inference01.csv'  # 저장할 경로 설정
df_preds.to_csv(csv_file_path, encoding='utf-8', index=False)  # 인덱스를 제외하고 csv 파일로 저장

csv_file_path  # 생성된 파일의 경로를 출력합니다.


'/mnt/h/내 드라이브/chatbot_project/Data/QLoRA_1epoch_inference01.csv'

In [ ]:
!pip install sentence_transformers

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
from transformers.models.distilbert import DistilBertModel

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [ ]:
submit = pd.read_csv('/mnt/h/내 드라이브/chatbot_project/Data/sample_submission.csv')    #이건 데이콘에서 지정해준 양식인 듯
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.013084,0.018426,-0.024363,-0.011639,0.056299,-0.009782,0.000777,0.053897,0.018704,...,-0.009056,-0.049563,0.011434,-0.027712,-0.017133,0.030111,0.060064,-0.022574,-0.002650,0.074633
1,TEST_001,-0.052057,0.035334,0.005357,0.016947,0.100648,-0.007485,0.033472,-0.033347,-0.026774,...,-0.041716,-0.048221,0.042644,-0.037683,-0.053899,0.057223,0.006944,-0.065675,-0.013808,0.014600
2,TEST_002,-0.001562,-0.049804,-0.033641,-0.005308,0.115158,-0.025872,0.000034,-0.030385,0.064911,...,-0.044507,-0.027046,0.056083,-0.040303,0.000891,-0.016994,-0.044742,-0.020527,-0.069947,0.074468
3,TEST_003,-0.002760,-0.006562,0.017367,0.022299,0.053757,-0.011873,-0.047967,-0.014642,0.018928,...,-0.041279,-0.008739,0.030187,-0.047284,0.007936,0.035385,-0.033151,-0.037501,0.013308,0.055455
4,TEST_004,-0.000864,0.017627,0.008884,-0.007991,0.107253,-0.001987,0.054401,0.052559,-0.025180,...,-0.009394,-0.045741,0.012355,0.009633,-0.037977,0.011705,0.009040,0.046507,0.002428,0.026616


In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('/mnt/h/내 드라이브/chatbot_project/Data/리더보드제출submission파일/QLoRA_10epoch03.csv', index=False)